In [1]:
import requests
import pandas as pd
# from bs4 import BeautifulSoup
from selenium import webdriver
import os

In [2]:
url = 'https://www.streetcheck.co.uk/postcode/startingwith/e17'

In [3]:
r = requests.get(url)

In [4]:
r.status_code
# returns 403 error which means The HTTP 403 Forbidden client error status response code indicates that the server understood the request but refuses to authorize it. 

403

In [5]:
# os.path.abspath(os.curdir)

In [6]:
# os.path.dirname(os.path.abspath(os.curdir))

In [7]:
# os.listdir(os.path.dirname(os.path.abspath(os.curdir)))

In [8]:
# want to iterate through several root files if to find  
# Development folder which has the chrome driver in it 

is_folder = False
# dir_list = None
start = os.path.dirname(os.path.abspath(os.curdir))
chrome_driver_path= None
while not is_folder:
    if 'Development' in os.listdir(start):
        filepath=start
        is_folder=True
#         print(start)
#         print(os.listdir(start))
#         print(os.path.join(start,'Development'))
        chrome_driver_path = os.path.join(start,'Development', 'chromedriver')
#         print(chrome_driver_path)
        if 'chromedriver' in os.listdir(os.path.dirname(chrome_driver_path)):
            print('chrome driver found')        
    else:
        start=os.path.dirname(start)

chrome driver found


In [9]:
driver = webdriver.Chrome(executable_path=chrome_driver_path)

OSError: [Errno 86] Bad CPU type in executable: '/Users/makaibe/Dropbox/My Mac (Nwamakas-MacBook-Pro.local)/Documents/Development/chromedriver'